# Model Context Protocol (MCP)

In diesem Notebook werden wir verschiedene Techniken zum Kontextmanagement in LLMs erkunden. Wir werden verschiedene Memory-Typen in LangChain implementieren und vergleichen.

## 1. Grundlagen des Kontextmanagements

Beim Arbeiten mit LLMs ist das Kontextmanagement entscheidend, da alle Modelle ein begrenztes Kontextfenster haben. Dieses Fenster begrenzt, wie viel Information das Modell bei einer Anfrage berücksichtigen kann.

Hier einige wichtige Konzepte:
- **Kontextfenster**: Die maximale Anzahl an Tokens, die ein Modell verarbeiten kann
- **Token**: Eine Grundeinheit der Textverarbeitung (etwa 4 Zeichen im Englischen)
- **Memory**: Mechanismus zur Speicherung relevanter Teile einer Konversation

In [1]:
# Benötigte Bibliotheken importieren
import os
from dotenv import load_dotenv
import time

# LangChain Importe
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Memory-Typen
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import VectorStoreRetrieverMemory
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Umgebungsvariablen laden
load_dotenv()

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'


# LLM initialisieren
llm = ChatOpenAI(model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

## 2. Kontextfenster verschiedener Modelle

Verschiedene LLMs haben unterschiedlich große Kontextfenster:

| Modell            | Kontextfenster | Besonderheiten                    |
|-------------------|----------------|-----------------------------------|
| GPT-3.5 Turbo     | 16K Tokens     | Kostengünstig                     |
| GPT-4 Turbo       | 128K Tokens    | Teurer, aber umfangreicherer Kontext |
| Claude 3 Opus     | 200K Tokens    | Sehr großes Kontextfenster        |
| Gemini Pro        | 32K Tokens     | Gutes Preis-Leistungs-Verhältnis  |
| Mistral Large     | 32K Tokens     | Open-Source Alternative           |

## 3. Token-Zählung verstehen

Lassen Sie uns untersuchen, wie Texte in Tokens umgewandelt werden. Dies ist wichtig, um das Kontextfenster effizient zu nutzen.

In [2]:
import tiktoken

def count_tokens(text, model="gpt-3.5-turbo"):
    """Zählt die Anzahl der Tokens in einem Text für ein bestimmtes Modell."""
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

def show_token_breakdown(text, model="gpt-3.5-turbo"):
    """Zeigt die einzelnen Tokens eines Textes an."""
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    decoded_tokens = [encoding.decode_single_token_bytes(token).decode("utf-8", errors="replace") for token in tokens]
    return tokens, decoded_tokens

# Beispieltexte
text1 = "KI ist eine tolle Sache."
text2 = "Künstliche Intelligenz revolutioniert viele Bereiche unseres Lebens."

# Token-Anzahl ausgeben
print(f"Text 1: '{text1}'")
print(f"Anzahl Tokens: {count_tokens(text1)}")
print(f"Text 2: '{text2}'")
print(f"Anzahl Tokens: {count_tokens(text2)}")

# Token-Breakdown für ersten Text
tokens, decoded = show_token_breakdown(text1)
print("\nToken-Breakdown für Text 1:")
for i, (token, decoded_token) in enumerate(zip(tokens, decoded)):
    print(f"Token {i+1}: {token} -> '{decoded_token}'")

Text 1: 'KI ist eine tolle Sache.'
Anzahl Tokens: 8
Text 2: 'Künstliche Intelligenz revolutioniert viele Bereiche unseres Lebens.'
Anzahl Tokens: 16

Token-Breakdown für Text 1:
Token 1: 82071 -> 'KI'
Token 2: 6127 -> ' ist'
Token 3: 10021 -> ' eine'
Token 4: 39674 -> ' tol'
Token 5: 273 -> 'le'
Token 6: 328 -> ' S'
Token 7: 1815 -> 'ache'
Token 8: 13 -> '.'


## 4. Memory-Typen in LangChain

LangChain bietet verschiedene Memory-Typen, die für unterschiedliche Anwendungsfälle optimiert sind.

### 4.1 Buffer Memory

Der einfachste Typ - speichert alle Nachrichten in voller Länge.

In [3]:
buffer_memory = ConversationBufferMemory()

# Konversation hinzufügen
buffer_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
buffer_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
buffer_memory.save_context({"input": "Was ist mein Name?"}, {"output": "Dein Name ist Hans."})

# Abrufen des gesamten Kontexts
context = buffer_memory.load_memory_variables({})
print(context["history"])

Human: Mein Name ist Hans
AI: Hallo Hans!
Human: Ich komme aus Berlin
AI: Berlin ist eine schöne Stadt.
Human: Was ist mein Name?
AI: Dein Name ist Hans.


/tmp/ipykernel_51918/800293615.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory()


### 4.2 Summary Memory

Erstellt eine fortlaufende Zusammenfassung der Konversation.

In [4]:
summary_memory = ConversationSummaryMemory(llm=llm)

# Konversation hinzufügen
summary_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
summary_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
summary_memory.save_context({"input": "Was ist mein Name?"}, {"output": "Dein Name ist Hans."})
summary_memory.save_context({"input": "Ich interessiere mich für künstliche Intelligenz."}, 
                          {"output": "Das ist ein spannendes Thema! KI umfasst viele Bereiche wie Machine Learning, neuronale Netze und natürliche Sprachverarbeitung."})

# Zusammenfassung abrufen
summary = summary_memory.load_memory_variables({})
print("Zusammenfassung der Konversation:")
print(summary["history"])

/tmp/ipykernel_51918/2706353524.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm)


Zusammenfassung der Konversation:
Hans from Berlin is introduced. The AI responds with a friendly greeting and acknowledges the beauty of Berlin. Hans asks "What is my name?" and the AI responds, "Your name is Hans." The human expresses interest in artificial intelligence in German, to which the AI responds enthusiastically, mentioning areas such as Machine Learning, neural networks, and natural language processing.


### 4.3 Window Memory

Behält nur die letzten n Nachrichten bei.

In [5]:
window_memory = ConversationBufferWindowMemory(k=2)  # Nur die letzten 2 Nachrichten behalten

# Konversation hinzufügen
window_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
window_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
window_memory.save_context({"input": "Was ist mein Name?"}, {"output": "Dein Name ist Hans."})

# Nur die letzten 2 Nachrichten abrufen
window_context = window_memory.load_memory_variables({})
print("Window Memory (letzte 2 Nachrichten):")
print(window_context["history"])

Window Memory (letzte 2 Nachrichten):
Human: Ich komme aus Berlin
AI: Berlin ist eine schöne Stadt.
Human: Was ist mein Name?
AI: Dein Name ist Hans.


/tmp/ipykernel_51918/2547049578.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=2)  # Nur die letzten 2 Nachrichten behalten


### 4.4 Token Buffer Memory

Begrenzt die Anzahl der gespeicherten Tokens.

In [6]:
token_memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

# Längere Konversation hinzufügen
token_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
token_memory.save_context({"input": "Erzähl mir etwas über künstliche Intelligenz"}, 
                        {"output": "Künstliche Intelligenz (KI) ist ein Teilgebiet der Informatik, das sich mit der Automatisierung intelligenten Verhaltens befasst. KI-Systeme können lernen, Probleme lösen und Entscheidungen treffen."})
token_memory.save_context({"input": "Was sind Anwendungsbeispiele?"}, 
                        {"output": "Anwendungsbeispiele für KI sind Spracherkennung, autonomes Fahren, medizinische Diagnose, Bilderkennung und natürlich Sprachmodelle wie ich."})

# Token-begrenzte Geschichte abrufen
token_context = token_memory.load_memory_variables({})
print("Token Buffer Memory (max. 100 Tokens):")
print(token_context["history"])

# Anzahl der Tokens in der Geschichte
print(f"\nAnzahl Tokens im Memory: {count_tokens(token_context['history'])}")

Token Buffer Memory (max. 100 Tokens):
Human: Was sind Anwendungsbeispiele?
AI: Anwendungsbeispiele für KI sind Spracherkennung, autonomes Fahren, medizinische Diagnose, Bilderkennung und natürlich Sprachmodelle wie ich.

Anzahl Tokens im Memory: 54


/tmp/ipykernel_51918/2684029810.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  token_memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)


### 4.5 Vector Store Memory

Speichert die Konversation in einer Vektordatenbank und ruft die relevantesten Nachrichten basierend auf der aktuellen Anfrage ab.

In [7]:
# Einfache Vektordatenbank erstellen
vectorstore = Chroma(embedding_function=embeddings, collection_name="memory_store")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Vector Memory initialisieren
vector_memory = VectorStoreRetrieverMemory(retriever=retriever)

# Verschiedene Informationen hinzufügen
vector_memory.save_context({"input": "Mein Name ist Hans"}, {"output": "Hallo Hans!"})
vector_memory.save_context({"input": "Ich komme aus Berlin"}, {"output": "Berlin ist eine schöne Stadt."})
vector_memory.save_context({"input": "Meine Lieblingsfarbe ist blau"}, {"output": "Blau ist eine schöne Farbe."})
vector_memory.save_context({"input": "Ich mag Pizza und Pasta"}, {"output": "Italienisches Essen ist sehr beliebt."})
vector_memory.save_context({"input": "Ich arbeite als Softwareentwickler"}, {"output": "Softwareentwicklung ist ein spannendes Berufsfeld."})

# Relevanten Kontext für eine Anfrage abrufen
query_info = vector_memory.load_memory_variables({"prompt": "Was war noch mal mein Name und woher komme ich?"})
print("Vector Memory (relevante Nachrichten basierend auf der Abfrage):")
print(query_info["history"])

/tmp/ipykernel_51918/2368775854.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings, collection_name="memory_store")
/tmp/ipykernel_51918/2368775854.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  vector_memory = VectorStoreRetrieverMemory(retriever=retriever)


Vector Memory (relevante Nachrichten basierend auf der Abfrage):
input: Mein Name ist Hans
output: Hallo Hans!
input: Ich komme aus Berlin
output: Berlin ist eine schöne Stadt.
input: Meine Lieblingsfarbe ist blau
output: Blau ist eine schöne Farbe.


## 5. Memory-Typen in der Praxis vergleichen

In [8]:
def create_conversation_chain(memory, name):
    """Erstellt eine Konversationskette mit dem angegebenen Memory-Typ."""
    return ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=True
    )

# Verschiedene Memory-Typen erstellen
buffer_memory = ConversationBufferMemory()
summary_memory = ConversationSummaryMemory(llm=llm)
window_memory = ConversationBufferWindowMemory(k=2)

# Konversationsketten erstellen
buffer_chain = create_conversation_chain(buffer_memory, "Buffer Memory")
summary_chain = create_conversation_chain(summary_memory, "Summary Memory")
window_chain = create_conversation_chain(window_memory, "Window Memory")

# Funktion zum Ausführen und Messen einer Konversation
def run_conversation(chain, name, messages):
    """Führt eine Konversation mit der angegebenen Kette durch und misst die Antwortzeit."""
    print(f"\n=== {name} ===\n")
    responses = []
    times = []
    
    for i, message in enumerate(messages):
        print(f"Nachricht {i+1}: {message}")
        start_time = time.time()
        response = chain.predict(input=message)
        end_time = time.time()
        elapsed = end_time - start_time
        
        responses.append(response)
        times.append(elapsed)
        
        print(f"Antwort: {response}")
        print(f"Zeit: {elapsed:.2f} Sekunden\n")
    
    return responses, times

/tmp/ipykernel_51918/84099166.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  return ConversationChain(


In [9]:
# Testreihe mit verschiedenen Fragen
test_messages = [
    "Mein Name ist Maria und ich komme aus München.",
    "Ich bin 35 Jahre alt und arbeite als Ärztin.",
    "In meiner Freizeit spiele ich gerne Klavier.",
    "Ich habe zwei Kinder, Thomas und Lisa.",
    "Kannst du mir sagen, wie ich heiße und was ich beruflich mache?"
]

# Konversation mit verschiedenen Memory-Typen durchführen
buffer_responses, buffer_times = run_conversation(buffer_chain, "Buffer Memory", test_messages)
summary_responses, summary_times = run_conversation(summary_chain, "Summary Memory", test_messages)
window_responses, window_times = run_conversation(window_chain, "Window Memory", test_messages)


=== Buffer Memory ===

Nachricht 1: Mein Name ist Maria und ich komme aus München.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Mein Name ist Maria und ich komme aus München.
AI:

> Finished chain.
Antwort: Hallo Maria! Schön, dich kennenzulernen. München ist eine sehr schöne Stadt in Deutschland. Hast du schon lange dort gelebt?
Zeit: 0.69 Sekunden

Nachricht 2: Ich bin 35 Jahre alt und arbeite als Ärztin.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

C

## 6. Vergleich der Antwortqualität

In [10]:
# Vergleich der letzten Antworten
print("Vergleich der letzten Antworten (Frage: Kannst du mir sagen, wie ich heiße und was ich beruflich mache?)")
print("\nBuffer Memory Antwort:")
print(buffer_responses[-1])
print("\nSummary Memory Antwort:")
print(summary_responses[-1])
print("\nWindow Memory Antwort:")
print(window_responses[-1])

# Vergleich der Antwortzeiten
print("\nDurchschnittliche Antwortzeiten:")
print(f"Buffer Memory: {sum(buffer_times)/len(buffer_times):.2f} Sekunden")
print(f"Summary Memory: {sum(summary_times)/len(summary_times):.2f} Sekunden")
print(f"Window Memory: {sum(window_times)/len(window_times):.2f} Sekunden")

Vergleich der letzten Antworten (Frage: Kannst du mir sagen, wie ich heiße und was ich beruflich mache?)

Buffer Memory Antwort:
Natürlich, Maria! Dein Name ist Maria und du arbeitest als Ärztin in München. Es ist schön, dich kennenzulernen und mehr über dich zu erfahren.

Summary Memory Antwort:
Natürlich! Du hast dich als Maria vorgestellt und gesagt, dass du Ärztin bist.

Window Memory Antwort:
Leider habe ich keine Informationen über deinen Namen oder deinen Beruf, Maria. Aber wenn du mir mehr darüber erzählen möchtest, bin ich hier, um zuzuhören und zu lernen. Gerne kannst du mir mehr über dich erzählen, damit ich dich besser kennenlernen kann.

Durchschnittliche Antwortzeiten:
Buffer Memory: 0.80 Sekunden
Summary Memory: 2.70 Sekunden
Window Memory: 1.26 Sekunden


## 7. Manuelle Implementation eines hierarchischen Gedächtnisses

Bei komplexen Anwendungen kann ein hierarchischer Ansatz sinnvoll sein, der verschiedene Memory-Typen kombiniert.

In [11]:
class HierarchicalMemory:
    def __init__(self, llm):
        # Kurzzeit-Gedächtnis: die letzten n Nachrichten (Window Memory)
        self.short_term = ConversationBufferWindowMemory(k=3)
        
        # Mittelfristiges Gedächtnis: Zusammenfassung bisheriger Konversation
        self.medium_term = ConversationSummaryMemory(llm=llm)
        
        # Langzeit-Gedächtnis: Wichtige Fakten in strukturierter Form
        self.long_term_facts = {}
        
        self.llm = llm
    
    def save_context(self, input_text, output_text):
        # Speichern im Kurzzeit- und mittelfristigen Gedächtnis
        self.short_term.save_context({"input": input_text}, {"output": output_text})
        self.medium_term.save_context({"input": input_text}, {"output": output_text})
        
        # Wichtige Fakten extrahieren und im Langzeitgedächtnis speichern
        self.extract_facts(input_text)
    
    def extract_facts(self, input_text):
        # Einfache Faktenextraktion durch Schlüsselwörter
        if "mein Name ist" in input_text.lower():
            parts = input_text.split("mein Name ist", 1)
            if len(parts) > 1:
                name = parts[1].split(".")[0].strip()
                self.long_term_facts["name"] = name
        
        if "ich komme aus" in input_text.lower():
            parts = input_text.split("ich komme aus", 1)
            if len(parts) > 1:
                location = parts[1].split(".")[0].strip()
                self.long_term_facts["location"] = location
        
        if "ich arbeite als" in input_text.lower():
            parts = input_text.split("ich arbeite als", 1)
            if len(parts) > 1:
                job = parts[1].split(".")[0].strip()
                self.long_term_facts["job"] = job
    
    def get_relevant_context(self, query):
        # Kontext aus verschiedenen Quellen zusammenstellen
        context = ""
        
        # 1. Wichtige Fakten aus dem Langzeitgedächtnis
        if self.long_term_facts:
            context += "Wichtige Fakten:\n"
            for key, value in self.long_term_facts.items():
                context += f"- {key.capitalize()}: {value}\n"
        
        # 2. Zusammenfassung der bisherigen Konversation
        medium_context = self.medium_term.load_memory_variables({})
        if medium_context["history"]:
            context += "\nZusammenfassung der Konversation:\n"
            context += medium_context["history"]
        
        # 3. Letzte Nachrichten aus dem Kurzzeitgedächtnis
        short_context = self.short_term.load_memory_variables({})
        if short_context["history"]:
            context += "\nLetzte Nachrichten:\n"
            context += short_context["history"]
        
        return context

In [12]:
# Hierarchisches Gedächtnis testen
hierarchical_memory = HierarchicalMemory(llm)

# Konversation simulieren
conversation = [
    ("Mein Name ist Klaus Schmidt.", "Hallo Klaus, schön dich kennenzulernen!"),
    ("Ich komme aus Hamburg.", "Hamburg ist eine schöne Stadt am Wasser."),
    ("Ich arbeite als Softwareentwickler.", "Softwareentwicklung ist ein spannender Beruf mit vielen Möglichkeiten."),
    ("Ich interessiere mich für künstliche Intelligenz.", "Das Gebiet der KI entwickelt sich rasant und bietet viele interessante Anwendungsmöglichkeiten."),
    ("Ich habe letztes Jahr ein Projekt mit Python umgesetzt.", "Python ist eine großartige Sprache für KI-Projekte und viele andere Anwendungen."),
]

for input_text, output_text in conversation:
    hierarchical_memory.save_context(input_text, output_text)
    print(f"User: {input_text}")
    print(f"AI: {output_text}\n")

# Relevanten Kontext für eine Anfrage abrufen
query = "Wie heißt du und was machst du beruflich?"
context = hierarchical_memory.get_relevant_context(query)
print("=== Hierarchisches Gedächtnis ===\n")
print(context)

# Langzeitgedächtnis überprüfen
print("\n=== Langzeitgedächtnis (extrahierte Fakten) ===\n")
for key, value in hierarchical_memory.long_term_facts.items():
    print(f"{key.capitalize()}: {value}")

User: Mein Name ist Klaus Schmidt.
AI: Hallo Klaus, schön dich kennenzulernen!

User: Ich komme aus Hamburg.
AI: Hamburg ist eine schöne Stadt am Wasser.

User: Ich arbeite als Softwareentwickler.
AI: Softwareentwicklung ist ein spannender Beruf mit vielen Möglichkeiten.

User: Ich interessiere mich für künstliche Intelligenz.
AI: Das Gebiet der KI entwickelt sich rasant und bietet viele interessante Anwendungsmöglichkeiten.

User: Ich habe letztes Jahr ein Projekt mit Python umgesetzt.
AI: Python ist eine großartige Sprache für KI-Projekte und viele andere Anwendungen.

=== Hierarchisches Gedächtnis ===


Zusammenfassung der Konversation:
Klaus Schmidt introduces himself to the AI, and they exchange greetings. Klaus mentions he is from Hamburg, and the AI responds by saying it's a beautiful city by the water. The human shares that they work as a software developer and mentions completing a project with Python last year. The AI responds by saying Python is a great language for AI proje

## 8. Praktische Anwendung: Chat mit effizientem Kontextmanagement

In [13]:
def create_optimized_chat():
    # Für einen effizienten Chat kombinieren wir verschiedene Memory-Typen
    
    # System-Prompt definieren
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Du bist ein hilfreicher Assistent. Verwende die folgenden Informationen, um Fragen zu beantworten:\n{context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    # Hierarchisches Gedächtnis für effizienten Kontext
    memory = HierarchicalMemory(llm)
    
    # Chat-Funktion erstellen
    def chat(message):
        context = memory.get_relevant_context(message)
        
        # Konversationshistorie aus dem Kurzzeitgedächtnis laden
        history = memory.short_term.load_memory_variables({})["history"]
        
        # LLM-Anfrage stellen
        messages = [
            SystemMessage(content=f"Du bist ein hilfreicher Assistent. Verwende die folgenden Informationen, um Fragen zu beantworten:\n{context}"),
            HumanMessage(content=message)
        ]
        response = llm.invoke(messages).content
        
        # Kontext aktualisieren
        memory.save_context(message, response)
        
        return response
    
    return chat

# Chat-Funktion erstellen
chat = create_optimized_chat()

# Beispiel-Konversation durchführen
messages = [
    "Hallo! Ich bin Julia und komme aus München.",
    "Ich bin 32 Jahre alt und arbeite als Grafikdesignerin.",
    "Ich habe einen Hund namens Bello und eine Katze namens Mimi.",
    "In meiner Freizeit gehe ich gerne wandern und fotografieren.",
    "Kannst du mir zusammenfassen, was du über mich weißt?"
]

for message in messages:
    print(f"\nUser: {message}")
    response = chat(message)
    print(f"AI: {response}")


User: Hallo! Ich bin Julia und komme aus München.
AI: Hallo Julia! Wie kann ich dir heute helfen?

User: Ich bin 32 Jahre alt und arbeite als Grafikdesignerin.
AI: Das klingt nach einem spannenden Beruf! Wenn du Fragen zu Grafikdesign hast oder Unterstützung bei einem bestimmten Projekt benötigst, lass es mich gerne wissen. Wie kann ich dir weiterhelfen?

User: Ich habe einen Hund namens Bello und eine Katze namens Mimi.
AI: Das klingt nach zwei süßen Haustieren! Hast du vielleicht Lust, ein Grafikdesignprojekt rund um deine Haustiere zu gestalten? Zum Beispiel ein Poster, eine Illustration oder ein Logo mit ihren Namen und Bildern? Lass mich wissen, wenn du dabei Unterstützung brauchst oder weitere Ideen hast.

User: In meiner Freizeit gehe ich gerne wandern und fotografieren.
AI: Das klingt nach tollen Hobbys! Wenn du möchtest, könnte man auch ein Grafikdesignprojekt gestalten, das deine Leidenschaft fürs Wandern und Fotografieren widerspiegelt. Zum Beispiel könntest du ein Naturpos

## 9. Best Practices für Kontextmanagement

1. **Relevanz priorisieren**: Nicht jede Information ist gleich wichtig. Priorisieren Sie kritische Informationen wie Nutzerdaten, Präferenzen und aktuelle Anfragen.

2. **Hierarchischen Ansatz verwenden**: Kombinieren Sie verschiedene Memory-Typen für verschiedene Informationsebenen:
   - **Kurzzeitgedächtnis**: Aktuelle Konversation (letzte 3-5 Nachrichten)
   - **Mittelfristiges Gedächtnis**: Zusammenfassung der bisherigen Konversation
   - **Langzeitgedächtnis**: Wichtige Fakten und Präferenzen

3. **Kontext regelmäßig komprimieren**: Verwenden Sie Summary Memory oder LLM-basierte Zusammenfassungen, um lange Konversationen zu komprimieren.

4. **Dynamische Kontexterweiterung**: Laden Sie nur dann externe Informationen, wenn sie für die aktuelle Anfrage relevant sind.

5. **Token-Überwachung**: Überwachen Sie die Anzahl der verwendeten Tokens, um Kosten zu kontrollieren und das Kontextfenster optimal zu nutzen.

6. **Metadaten nutzen**: Speichern Sie Zeitstempel und Wichtigkeitsbewertungen für jede Information, um später besser filtern zu können.

7. **Vergessen implementieren**: Entfernen Sie weniger relevante oder veraltete Informationen aus dem Kontext.

## 10. Fazit

Effektives Kontextmanagement ist entscheidend für die Leistung von LLM-Anwendungen. Die richtige Strategie hängt vom konkreten Anwendungsfall ab:

- **Buffer Memory**: Gut für kurze Konversationen mit vollständigem Kontext
- **Summary Memory**: Sinnvoll für längere Konversationen, bei denen Details weniger wichtig sind
- **Window Memory**: Effizient für aufgabenorientierte Dialoge, wo nur der aktuelle Kontext wichtig ist
- **Vector Memory**: Ideal für sehr lange Konversationen oder große Wissensdatenbanken
- **Hierarchischer Ansatz**: Die beste Lösung für komplexe Anwendungen mit unterschiedlichen Kontextanforderungen

Durch die richtige Kombination dieser Techniken können Sie die Effektivität Ihrer LLM-basierten Anwendungen maximieren und gleichzeitig Kosten und Latenz minimieren.